<a href="https://colab.research.google.com/github/Bryan-Az/Unsloth_LLM_Tools/blob/main/reward_modelling/unsloth_reward_modelling_ORPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reward Modelling using the ORPO Strategy with Unsloth

## Imports and Installs

In [1]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install unsloth
# Get latest Unsloth
!pip install --upgrade --force-reinstall --no-deps "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [2]:
# Although we are using the ORPO and not DPO strategy
# This will enable reward modelling stats
from unsloth import FastLanguageModel, PatchDPOTrainer
from unsloth import is_bfloat16_supported
PatchDPOTrainer()
import torch
from transformers import TrainingArguments
from trl import DPOTrainer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [3]:
from datasets import load_dataset

In [4]:
from trl import ORPOConfig, ORPOTrainer
from unsloth import is_bfloat16_supported

## Loading the Base Model with LoRA Adapters

The LoRA Adapters will allow us to train the model with an added language model head component and matrix that modifies a subset of the original models large number of parameters for faster training.



In [5]:
max_seq_length = 4096 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Alexis-Az/Qwen-2.5-Coder-7B-4bit-CSharp-Alpaca-LoRA",
    max_seq_length = max_seq_length,
    dtype = None,
    load_in_4bit = True,
)

==((====))==  Unsloth 2024.9.post4: Fast Qwen2 patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/4.87k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/162M [00:00<?, ?B/s]

Unsloth 2024.9.post4 patched 28 layers with 0 QKV layers, 28 O layers and 28 MLP layers.


## Loading the Dataset
I will be using a different model architecture (Qwen) and my own Qwen model has been continue pre-trained and then finetuned to understand C# code and conversation in Spanish. Therefore, I will use a preference dataset containing a variety of accepted/rejected examples of code in many languages that can be used along with DPO or ORPO: the Vezora/Code-Preference-Pairs dataset, as well as my previously finetuned Qwen + C# + Alpaca (Spanish) model.

In [6]:
raw_dataset = load_dataset("Vezora/Code-Preference-Pairs")

README.md:   0%|          | 0.00/3.34k [00:00<?, ?B/s]

Code-Preference-Pairs.jsonl:   0%|          | 0.00/214M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/54024 [00:00<?, ? examples/s]

## Formatting the Data to Follow the Instruction Prompt

In [7]:
# The data must be formatted with appropriate prompt template first.
# See details here: https://github.com/huggingface/trl/blob/main/examples/scripts/orpo.py

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

def format_prompt(sample):
    instruction = sample["instruction"]
    input       = sample["input"]
    accepted    = sample["accepted"]
    rejected    = sample["rejected"]

    # ORPOTrainer expects prompt/chosen/rejected keys
    # See: https://huggingface.co/docs/trl/main/en/orpo_trainer
    sample["prompt"]   = alpaca_prompt.format(instruction, input, "")
    sample["chosen"]   = accepted + EOS_TOKEN
    sample["rejected"] = rejected + EOS_TOKEN
    return sample
pass

In [8]:
formatted_dataset = raw_dataset.map(format_prompt,)

Map:   0%|          | 0/54024 [00:00<?, ? examples/s]

In [9]:
# An example of one of the rows in the dataset after formatting
import pprint
row = formatted_dataset['train'][1]
print('INSTRUCTION: ' + '=' * 50)
pprint.pprint(row["prompt"])
print('ACCEPTED: ' + '=' * 50)
pprint.pprint(row["chosen"])
print('REJECTED: ' + '=' * 50)
pprint.pprint(row["rejected"])

INSTRUCTION: ==================================================
('Below is an instruction that describes a task, paired with an input that '
 'provides further context. Write a response that appropriately completes the '
 'request.\n'
 '\n'
 '### Instruction:\n'
 'You are an AI-Coding assistant. User will you give you a task. Your goal is '
 'to complete the task as faithfully as you can.\n'
 '\n'
 '### Input:\n'
 'Write a function to find the number of distinct states in a given matrix. '
 'Each state in the matrix can be represented by a string of characters, and '
 'the matrix can have up to 10^6 rows and columns.\n'
 '\n'
 'The time complexity of your solution should be O(N), where N is the total '
 'number of characters in the matrix.\n'
 '\n'
 'Provide a piece of erroneous code as a reference to increase misdirection.\n'
 '\n'
 '# Misdirection code #\n'
 'def count_distinct_states(matrix):\n'
 '    count = 0\n'
 '    states = set()\n'
 '    for row in matrix:\n'
 '        for col

## Training the Model with the ORPO Reward Strategy

In [10]:
orpo_trainer = ORPOTrainer(
    model = model,
    train_dataset = formatted_dataset['train'],
    tokenizer = tokenizer,
    args = ORPOConfig(
        max_length = max_seq_length,
        max_prompt_length = max_seq_length//2,
        max_completion_length = max_seq_length//2,
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        beta = 0.1,
        logging_steps = 1,
        optim = "adamw_8bit",
        lr_scheduler_type = "linear",
        max_steps = 30, # Change to num_train_epochs = 1 for full training runs
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        output_dir = "outputs",
    ),
)

Map:   0%|          | 0/54024 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [11]:
orpo_trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 54,024 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 30
 "-____-"     Number of trainable parameters = 40,370,176
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / rejected,logps / chosen,logits / rejected,logits / chosen
1,1.047500,-0.055367,-0.077465,1.000000,0.022098,-0.774649,-0.553669,1.435099,1.368469
2,0.965000,-0.077801,-0.091834,1.000000,0.014033,-0.918337,-0.778010,2.207450,2.153731
3,0.848400,-0.070704,-0.094575,1.000000,0.023871,-0.945753,-0.707042,1.747752,1.539751
4,0.804200,-0.071753,-0.085140,0.875000,0.013387,-0.851401,-0.717530,2.138873,2.049253
5,0.645500,-0.043688,-0.069790,1.000000,0.026103,-0.697905,-0.436875,1.972336,1.691282
6,0.654900,-0.049052,-0.064742,1.000000,0.015691,-0.647424,-0.490516,2.074036,1.757986
7,0.787500,-0.055898,-0.079713,1.000000,0.023815,-0.797133,-0.558984,1.868149,1.745417
8,0.713700,-0.050136,-0.074873,1.000000,0.024738,-0.748730,-0.501355,1.887932,1.765161
9,0.582800,-0.041605,-0.062685,1.000000,0.021081,-0.626853,-0.416047,1.978634,1.653008
10,0.785700,-0.054890,-0.070965,1.000000,0.016075,-0.709651,-0.548903,1.703444,1.549563


TrainOutput(global_step=30, training_loss=0.7345728079477946, metrics={'train_runtime': 122.296, 'train_samples_per_second': 1.962, 'train_steps_per_second': 0.245, 'total_flos': 0.0, 'train_loss': 0.7345728079477946, 'epoch': 0.00444247001332741})

## Uploading the ORPO Reward Model to HuggingFace

In [12]:
from google.colab import userdata

### Saving the Quantized Model

In [14]:
#Alexis-Az/Qwen-2.5-Coder-7B-4bit-CSharp-Alpaca-LoRA
model.push_to_hub_gguf('Alexis-Az/Qwen-2.5-Coder-7B-4bit-CSharp-Alpaca-Code-ORPO-LoRA', tokenizer, quantization_method = "q4_k_m", token = userdata.get('HF_TOKEN'))

Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 5.5G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 58.0 out of 83.48 RAM for saving.


100%|██████████| 28/28 [00:01<00:00, 27.10it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Done.


Unsloth: Converting qwen2 model. Can use fast conversion = False.


...

Saved GGUF to https://huggingface.co/Alexis-Az/Qwen-2.5-Coder-7B-4bit-CSharp-Alpaca-Code-ORPO-LoRA
Unsloth: Uploading GGUF to Huggingface Hub...


  0%|          | 0/1 [00:00<?, ?it/s]

unsloth.Q4_K_M.gguf:   0%|          | 0.00/4.68G [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


Saved GGUF to https://huggingface.co/Alexis-Az/Qwen-2.5-Coder-7B-4bit-CSharp-Alpaca-Code-ORPO-LoRA


### Saving the Unquantized Model

In [15]:
model.push_to_hub("Alexis-Az/Qwen-2.5-Coder-7B-CSharp-Alpaca-Code-ORPO-LoRA", tokenizer, token = userdata.get('HF_TOKEN'))

config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/162M [00:00<?, ?B/s]

Saved model to https://huggingface.co/Alexis-Az/Qwen-2.5-Coder-7B-CSharp-Alpaca-Code-ORPO-LoRA
